In [1]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from tqdm import tqdm

In [40]:
test_x=[]
test_y=[]
train_x=[]
train_y=[]


In [17]:
def make1ch(img):
    lum_img = img[:,:,0]
    return lum_img 

In [ ]:

#res  = Image.open('C:\\Users\\youssef mahmoud\\desktop\\sub-block-poster.png')
#res.crop((0,0,5,5))
#res.resize((512,512))
#res.show()
"""
im = imageio.imread('C:\\Users\\youssef mahmoud\\desktop\\Allsubs.png')
#im= np.array(res)
im = make1ch(im)
im = np.array(im)

im = (im>60)*1
#im = (im==0)*1
pred = []

for i in range(4):
    for j in range(4):
        pred.append(im[j*512:(j+1)*512,i*512:(i+1)*512].reshape(512,512,1))
        
        
unique_elements, counts_elements = np.unique(new, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))
plt.imshow(new)
plt.gray()
plt.show()
    
pred = np.array(pred)
print(pred.shape)
pred = pred.reshape(16,512,512,1)
pred = np.float32(pred)
"""

## loading the data

In [ ]:

train_x = []
for i in tqdm(range (0,4000)):
    im = imageio.imread('SegData5\\Images\\'+str(i)+'.png')
    im = make1ch(im)
    im = np.array(im)
    im = im.reshape(512,512,1)
    train_x.append(im)
train_x = np.array(train_x)
#train_x[train_x>170] = 0
train_x = (train_x>253.0)*1
train_x = np.float32(train_x)



In [ ]:
train_y = []
for i in tqdm(range (0,4000)):
    im = imageio.imread('SegData5\\Labels\\'+str(i)+'.png')
    im = make1ch(im)
    im = np.array(im)
    im = im.reshape(512,512,1)
    train_y.append(im)
train_y = np.array(train_y)
#train_y[train_y>170] = 0
train_y = (train_y>253.0)*1
train_y = np.float32(train_y)

print (train_y.shape)

In [ ]:
test_x = []
for i in tqdm(range (10000,10500)):
    im = imageio.imread('SegData5\\Images\\'+str(i)+'.png')
    im = make1ch(im)
    im = np.array(im)
    im = im.reshape(512,512,1)
    test_x.append(im)
test_x = np.array(test_x)
#test_x[test_x>170] = 0
test_x = (test_x>253.0)*1.0
test_x = np.float32(test_x)
print (test_x.shape)

In [ ]:
test_y = []
for i in tqdm(range (10000,10500)):
    im = imageio.imread('SegData5\\Labels\\'+str(i)+'.png')
    im = make1ch(im)
    im = np.array(im)
    im = im.reshape(512,512,1)
    test_y.append(im)
test_y = np.array(test_y)
#test_y[test_y>170] = 0
test_y = (test_y>253.0)*1
test_y = np.float32(test_y)
print (test_y.shape)

## making & compiling the model

In [1]:
#import keras
from  tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.layers import Input, Dense, Activation
from tensorflow.keras.layers import concatenate  # functional interface
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LeakyReLU

N_INPUT = None


def get_unet():
    n_ch_exps = [6, 6, 7, 7, 8, 8]
    kernels = (5, 5)

    if K.image_data_format() == 'channels_first':
        ch_axis = 1
        input_shape = (1, N_INPUT, N_INPUT)
    elif K.image_data_format() == 'channels_last':
        ch_axis = 3
        input_shape = (N_INPUT, N_INPUT, 1)

    inp = Input(shape=input_shape)
    encodeds = []

    # encoder
    enc = inp
    for l_idx, n_ch in enumerate(n_ch_exps):
        enc = Conv2D(2 ** n_ch, kernels,
                     strides=(2, 2), padding='same',
                     kernel_initializer='he_normal')(enc)
        enc = LeakyReLU(name='encoded_{}'.format(l_idx),
                        alpha=0.2)(enc)
        encodeds.append(enc)

    # decoder
    dec = enc
    decoder_n_chs = n_ch_exps[::-1][1:]
    for l_idx, n_ch in enumerate(decoder_n_chs):
        l_idx_rev = len(n_ch_exps) - l_idx - 2  #
        dec = Conv2DTranspose(2 ** n_ch, kernels,
                              strides=(2, 2), padding='same',
                              kernel_initializer='he_normal',
                              activation='relu',
                              name='decoded_{}'.format(l_idx))(dec)
        dec = concatenate([dec, encodeds[l_idx_rev]],
                          axis=ch_axis)

    outp = Conv2DTranspose(1, kernels,
                           strides=(2, 2), padding='same',
                           kernel_initializer='glorot_normal',
                           activation='sigmoid',
                           name='decoded_{}'.format(l_idx + 1))(dec)

    unet = Model(inputs=inp, outputs=outp)
    return unet



In [33]:
model = get_unet()
opt =Adam(learning_rate=1e-4)
model.compile(optimizer = opt, loss = 'binary_crossentropy' , metrics = ['accuracy'])

In [ ]:
model.fit(train_x,train_y, epochs=4, batch_size=10)

In [ ]:
model.save('trained_model.h5')

## Evaluating the model

In [ ]:
results = model.evaluate(test_x, test_y, batch_size=1)

print("accuracy :" + str(results[1]*100)+"%")

## Visual testing

In [ ]:
#compare output of an image
import tensorflow as tf
pictureNum=1;

im = imageio.imread('SegData5\\Images\\'+str(pictureNum)+'.png')
im = make1ch(im)
im = np.array(im);
im = im.reshape(1,512,512,1)
im = (im>254.0)*1
im2 = tf.cast(im, tf.float32)
pred = model.predict(im2)
print(pred.shape)
imgLabel = imageio.imread('SegData5\\Labels\\'+str(pictureNum)+'.png')
imgLabel = make1ch(imgLabel)
imgLabel = np.array(imgLabel);
imgLabel = imgLabel.reshape(1,512,512,1)
imgLabel = (imgLabel>253)*1



In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

fig = plt.figure(figsize=(10, 8),dpi=300)

columns = 3
rows = 1
fig.add_subplot(rows, columns, 1)
plt.imshow(im[0].reshape(512,512))

fig.add_subplot(rows, columns, 2)
plt.imshow(imgLabel[0].reshape(512,512))

fig.add_subplot(rows, columns, 3)
m = pred[0]*255
#m = (m>190)*1
plt.imshow(m.reshape(512,512))

#plt.imshow(img)


plt.gray()
plt.show()

## loading trained model

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('trained_model.h5')